In [5]:
# add a new directory to the path
import sys
sys.path.append('../')

import nest_asyncio
import sklearn.feature_selection
import sklearn.model_selection  # Only necessary in Notebooks
import time
import sklearn.metrics
from collections.abc import Iterable
import pandas as pd
import sklearn
import numpy as np
from typing import Any

import openml
from sklearn.preprocessing import LabelEncoder

from amltk.sklearn import split_data

import numpy as np
import openml
from sklearn.compose import make_column_selector
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import (
    LabelEncoder,
    MinMaxScaler,
    OneHotEncoder,
    RobustScaler,
    StandardScaler,
)
from sklearn.svm import SVC

from amltk.data.conversions import probabilities_to_classes
from amltk.ensembling.weighted_ensemble_caruana import weighted_ensemble_caruana
from amltk.optimization import History, Metric, Trial
from amltk.optimization.optimizers.smac import SMACOptimizer
from amltk.pipeline import Choice, Component, Sequential, Split
from amltk.scheduling import Scheduler
from amltk.sklearn.data import split_data
from amltk.store import PathBucket
from amltk.optimization import Metric
from amltk.scheduling import Scheduler
from amltk.optimization.optimizers.smac import SMACOptimizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from amltk.pipeline import Component, Node, Sequential, Split
from functools import partial
import pandas as pd

pipeline = (
    Sequential(name="Pipeline")
    >> Split(
        {
            "categories": [
                SimpleImputer(strategy="constant", fill_value="missing"),
                Component(
                    OneHotEncoder,
                    space={
                        "min_frequency": (0.01, 0.1),
                        "handle_unknown": ["ignore", "infrequent_if_exist"],
                    },
                    config={"drop": "first"},
                ),
            ],
            "numbers": [
                Component(SimpleImputer, space={"strategy": ["mean", "median"]}),
                Component(VarianceThreshold, space={"threshold": (0.0, 0.2)}),
                Choice(StandardScaler, MinMaxScaler, RobustScaler, name="scaler"),
            ],
        },
        name="feature_preprocessing",
        config={
            "categories": make_column_selector(dtype_include=object),
            "numbers": make_column_selector(dtype_include=np.number),
        },
    )
    >> Choice(  # <!> (1)!
        Component(SVC, space={"C": (0.1, 10.0)}, config={"probability": True}),
        Component(
            RandomForestClassifier,
            space={"n_estimators": (10, 100), "criterion": ["gini", "log_loss"]},
        ),
        Component(
            MLPClassifier,
            space={
                "activation": ["identity", "logistic", "relu"],
                "alpha": (0.0001, 0.1),
                "learning_rate": ["constant", "invscaling", "adaptive"],
            },
        ),
    )
)

import nest_asyncio
import sklearn.feature_selection
import sklearn.model_selection  # Only necessary in Notebooks
nest_asyncio.apply()
scorers = ['accuracy']


X = np.random.rand(100,10)
y = np.random.randint(0,2,100)

X = pd.DataFrame(X)
y = pd.DataFrame(y)

In [6]:
import automl_estimators

cv = sklearn.model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
est = automl_estimators.AMLTKEstimator(pipeline=pipeline, scorers=scorers, other_objective_functions=[], 
                                      cv=cv, max_time_mins=5, n_jobs=1, seed=None, max_evals=2)

est.fit(X,y)

Output()

/home/perib/Projects/common/Projects/TPOT_Dev/automl_estimators/examples/../automl_estimators/amltk_estimator.py:178: UserWarning: Detected that current running context is in a notebook! When `display='auto'`, the default, the scheduler will automatically be set to display. If you do not want this or wish to disable this warning, please set `display=False`.
  scheduler.run(timeout=self.max_time_mins*60, wait=False)


Trial history:
                                                     status  trial_seed  \
name                                                                      
config_id=1_seed=2006673745_budget=None_instanc...     fail  2006673745   
config_id=2_seed=2006673745_budget=None_instanc...  success  2006673745   

                                                                                            exception  \
name                                                                                                    
config_id=1_seed=2006673745_budget=None_instanc...  No feature in X meets the variance threshold 0...   
config_id=2_seed=2006673745_budget=None_instanc...                                                 NA   

                                                                                            traceback  \
name                                                                                                    
config_id=1_seed=2006673745_budget=None_instanc...  T

/home/perib/miniconda3/envs/myenv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [7]:
est.fitted_pipeline_

Pipeline(steps=[('feature_preprocessing',
                 ColumnTransformer(transformers=[('categories',
                                                  Pipeline(steps=[('SimpleImputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 min_frequency=0.0896021893527))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x76f4...
                                                 ('numbers',
                                                  Pipeline(steps=[('SimpleImputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('VarianceThreshold',
                                                                   VarianceThreshold(threshold=0.0458378126845)),
                                                                  ('MinMaxScaler',
                                                                   MinMaxScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x76f4bf5001f0>)])),
                ('RandomForestClassifier',
                 RandomForestClassifier(criterion='log_loss',
                                        n_estimators=63))])

In [8]:
import automl_estimators

pipeline = (
    Sequential(name="Pipeline")
    >> Component(sklearn.preprocessing.PolynomialFeatures, space={"degree": (2,3)})
    >> Component(
            RandomForestClassifier,
            space={"n_estimators": (10, 100), "criterion": ["gini", "log_loss"]},
        )
)
cv = sklearn.model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
est = automl_estimators.AMLTKEstimator(pipeline=pipeline, scorers=scorers, other_objective_functions=[], 
                                      cv=cv, max_time_mins=5, n_jobs=1, seed=None, max_evals=2, backend='optuna')

est.fit(X,y)

[I 2024-10-14 15:26:11,042] A new study created in memory with name: no-name-ff34dae1-2431-4e57-a9fc-30a7f514d0bf


Output()

/home/perib/Projects/common/Projects/TPOT_Dev/automl_estimators/examples/../automl_estimators/amltk_estimator.py:178: UserWarning: Detected that current running context is in a notebook! When `display='auto'`, the default, the scheduler will automatically be set to display. If you do not want this or wish to disable this warning, please set `display=False`.
  scheduler.run(timeout=self.max_time_mins*60, wait=False)


Trial history:
                 status  trial_seed exception traceback  \
name                                                      
trial_number=0  success  1584723159        NA        NA   
trial_number=1  success  1584723159        NA        NA   

                                                           bucket  \
name                                                                
trial_number=0  OptunaOptimizer-2024-10-14T15:26:11.042096/tri...   
trial_number=1  OptunaOptimizer-2024-10-14T15:26:11.042096/tri...   

                               created_at                reported_at  \
name                                                                   
trial_number=0 2024-10-14 15:26:11.044614 2024-10-14 15:26:11.289895   
trial_number=1 2024-10-14 15:26:11.309921 2024-10-14 15:26:11.419648   

                metric:accuracy_score (maximize)  summary:accuracy_score  \
name                                                                       
trial_number=0                

/home/perib/miniconda3/envs/myenv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [9]:
est.predict(X)

array([1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1])